In [ ]:
                                                        #### Task/The problem  #######  
#  My client has some apartments he would like to sell, but he would like to know the best condition to initiate the sale, and best time of the year.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
from datetime import datetime, date, time, timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
##Step 1 is to load the data into python
housing_data = pd.read_csv('data/Housing_info.csv')
housing_data.head()

In [ ]:
housing_data.dtypes

In [ ]:
housing_data.columns

In [ ]:
housing_data.shape

In [ ]:
housing_data.duplicated().value_counts()

In [ ]:
housing_data.isna().sum()

In [ ]:
## My client has a series of 3 bedroom apartments to sell
 

In [ ]:
housing_step1 = housing_data[housing_data['bedrooms'].isin([2,3,4,5])]
housing_step1.head()

In [ ]:
### Now we have a collection of 2,3,4,5 bedrooms which we would use for robustness checks. As stated above , the client has 
### an interest in the 3 bedroom apartments. 

## To do this, we will take a look at the 3 bedroom apartments in our data which is the next step. 
## After extracting the data, we will start the first and important step which is the data cleaning.

In [ ]:
housing_3 = housing_step1[housing_step1['bedrooms'].isin([3])]
housing_3

In [ ]:
housing_3.dtypes

In [ ]:
### Change the data types of some variables

data_types_dict = {'bedrooms':'int', 'bathrooms':'int', 'floors': 'int',
                 'sqft_above': 'int', 'sqft_living': 'int',
                   'sqft_lot': 'int', 'sqft_lot15':'int','price': 'int'
                  
                  }

In [ ]:
housing_3 = housing_3.astype(data_types_dict)
housing_3.dtypes

In [ ]:
# check for missing values
housing_3.isna().sum()

In [ ]:
housing_3['date'] = pd.to_datetime(housing_3['date'], format='%Y-%m-%d')
housing_3.dtypes

In [ ]:
housing_3['sales_month'] = housing_3['date'].dt.strftime('%B')
housing_3.head()

In [ ]:
## generate a new year renovated column which will represent a categorical variable, with 0= no renovation, 1= renovation
housing_3.eval('year_renovated = yr_renovated', inplace=True)
housing_3.head()

In [ ]:
housing_3.dropna(subset=['yr_renovated'], inplace=True)
housing_3
# Why do i drop the N/a data? In this case, we are only interested in renovation with values, all missing entries will be dropped from the data 

In [ ]:
##Transform newly generated column into a categorical variable. 1 represents renovation at some point, 0 never been renovated
housing_3['year_renovated'] = np.where(housing_3['year_renovated'] > 0, 1, 0)

In [ ]:
## Question 1: WILL RENOVATING BEFORE SELLING BE BETTER THAN NOT?

In [ ]:
#Define Price Variable 
price = housing_3.groupby('year_renovated')['price'].mean()
price= price.astype(int)

renovation = housing_3['year_renovated'].unique()

data1 = {'price': price, 'renovation': renovation}

housing_3bed = pd.DataFrame(data1)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

colors = ['green', 'orange']
plt.figure(figsize=(8, 6))
sns.barplot(data=housing_3bed,x='renovation',y='price', palette=colors)

def currency_formatter(x, pos):
    return "${:,.0f}".format(x)

formatter = FuncFormatter(currency_formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.ylabel('Average price', fontweight='bold')
plt.xlabel('House renovation',fontweight='bold')
plt.title('Average price renovated vs non-renovated', fontweight='bold')


In [ ]:
## From the table above, we see houses renovated are on average are likely to sell at a higher mean price vs non renovated houses.
 


#Follow Up:
    ## Could the difference be accounted for by other factors?

# To test if there would be differences in our result if other factors are controlled for, we would account for Waterfront and grade. These 2 factors are expected to
# have an influence on prices.


In [ ]:
## Again we start with 3 bedroom apartments which is the apartment of interest
housing_3.head()

In [ ]:
housing_3.isna().sum()

In [ ]:
housing_2 = housing_3

In [ ]:
housing_grade = housing_2
housing_grade

In [ ]:
data_types_dict = {'grade': 'int'}

In [ ]:
 housing_grade.astype(data_types_dict)
housing_grade

In [ ]:
housing_grade.dtypes

In [ ]:
## For this part of our analysis we would repeat same step similar to year renovated. Generate Categorical variables
# Grade 0- 3: 0
# Grade >3 <= 6 :1
# Grade >6 <=9 :2
# Others == 3

housing_grade['grade'] = np.where(housing_grade['grade'] <= 3, 0,
                                np.where(housing_grade['grade'] <= 6, 1,
                                         np.where(housing_grade['grade'] <= 9, 2, 3)))
housing_grade['grade'].unique()

In [ ]:
renovation2 = housing_grade['year_renovated'].unique()
Interest = housing_grade.groupby(['year_renovated','grade'])['price'].mean().reset_index()
data4_types_dict = {'price': 'int'}
Interest = Interest.astype(data4_types_dict)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

plt.figure(figsize=(9, 6))


def currency_formatter(x, pos):
    return "${:,.0f}".format(x)

formatter = FuncFormatter(currency_formatter)
plt.gca().yaxis.set_major_formatter(formatter)
sns.barplot(data=Interest, x='grade',y='price', hue= 'year_renovated')

plt.xlabel('House renovation',fontweight = 'bold')
plt.title('Average price renovated vs non-renovated by grade',fontweight = 'bold')
sns.barplot(data=Interest, x='grade',y='price', hue= 'year_renovated', palette='Set2')


In [ ]:

## Even after grouping into grades, the result still holds true

## renovated houses on average sell higher than non-renovated houses.

In [ ]:
### Checking for waterfront differential

In [ ]:
house_water = housing_3

In [ ]:
house_water.isna().sum()

In [ ]:
house_water.dropna(subset='waterfront', inplace=True)

In [ ]:
house_water['waterfront'] = house_water.waterfront.astype('int')
house_water.head()

In [ ]:
Interest3 = house_water.groupby(['year_renovated','waterfront'])['price'].mean().reset_index()
data5_types_dict = {'price': 'int'}
Interest3 = Interest3.astype(data5_types_dict)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

plt.figure(figsize=(9, 6))


def currency_formatter(x, pos):
    return "${:,.0f}".format(x)

formatter = FuncFormatter(currency_formatter)
plt.gca().yaxis.set_major_formatter(formatter)


plt.xlabel('House renovation',fontweight='bold')
plt.title('Average price renovated vs non-renovated by waterfront', fontweight='bold')
sns.barplot(data=Interest3, x='waterfront',y='price', hue= 'year_renovated', palette='Set1')
plt.savefig('seaborn_plot.png')

In [ ]:
## Result:
## Even when you compare houses with a view, renovated houses on average still sell higher than non-renovated houses.

In [ ]:
## Task 2 when is the best time to sell ?

In [ ]:
Un = housing_3.groupby(['year_renovated','sales_month'])['price'].mean().reset_index()
plt.figure(figsize=(11,6))

sns.lineplot(data=Un, x= 'sales_month',y='price', hue='year_renovated')
plt.ylabel('Price in $', fontweight= 'bold')
plt.title('Best Month To Sell Renovated vs Non-renovated',fontweight= 'bold')


## Result
## If my client decides to renovate before selling, the best time to sell the house would be in December.

## Without renovation, April seems to be the logical time

In [ ]:
###
#                      Robustness check 
#Question 2: Does this result hold true for 2, 4, 5 bedrooms?

In [ ]:
housing_others = housing_data[housing_data['bedrooms'].isin([2,4,5])]
housing_others.head()

In [ ]:
housing_others['date'] = pd.to_datetime(housing_others['date'], format='%Y-%m-%d')
housing_others.dtypes

In [ ]:
housing_others['sales_month'] = housing_others['date'].dt.strftime('%B')
housing_others.head()

In [ ]:
data_types_dict = {'bedrooms':'int', 'bathrooms':'int', 'floors': 'int',
                 'sqft_above': 'int', 'sqft_living': 'int',
                   'sqft_lot': 'int', 'sqft_lot15':'int','price': 'int'
                  
                  }

In [ ]:
housing_others= housing_others.astype(data_types_dict)
housing_others.head()

In [ ]:
housing_others.eval('year_renovated = yr_renovated', inplace=True)

In [ ]:
housing_others.dropna(subset='yr_renovated', inplace=True)
housing_others.isna().sum()

In [ ]:
housing_others['year_renovated'] = np.where(housing_others['year_renovated'] > 0, 1, 0)

In [ ]:
price1= housing_others.groupby('year_renovated')['price'].mean()
price1= price1.astype(int)

renovation1 = housing_others['year_renovated'].unique()

data11 = {'price1': price1, 'renovation1': renovation1}

House_other = pd.DataFrame(data11)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

plt.figure(figsize=(6, 6))
sns.barplot(data=House_other,x='renovation1',y='price1')

def currency_formatter(x, pos):
    return "${:,.0f}".format(x)

formatter = FuncFormatter(currency_formatter)
plt.gca().yaxis.set_major_formatter(formatter)

plt.xlabel('House renovation',fontweight ='bold')
plt.title('Average price renovated vs non-renovated',fontweight ='bold')


In [ ]:
## Ans: Result derived from our earlier analysis appear true for when you check apartments with 2,4,5 bedrooms.

In [ ]:
## Geographical checks

In [ ]:
hhh3 = housing_3[housing_3['bathrooms'].isin([3])]

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(hhh3, lat='lat', lon='long',color='year_renovated', size='price',
                 zoom=3, mapbox_style='carto-positron')
fig.update_layout(width=1000, height=800)
fig.show()
fig.write_html('interactive_map.html')

In [ ]:
## Conclusion 

# 1. It would appear renovating apartments increases the selling price on average even. ( This finding holds for other houses not in 
# clients portfolio( 2,4,5)
# 2. Although an increase is noticed even in houses in grade 1, renovating or not renovating doesn't appear to make a difference 

# In summary, without having information about the cost of renovation, it would appear that renovating the apartments would be in my clients interest especially 
# if said apartment is within grade 2 and 3. 

# If my client proceeds with renovating, best time to initiate sale would be in december

# It must be emphasied that the cost of renovation must be factored in before any decision is made. Overall, my client can command a higher selling price if 
# he choses to renovate. But it must be highlighted that in some cases(Grade 1) the difference noticed could potentially be easily offset by renovation expenses,
# it would not make financial sense to renovate.
# The finding also holds when you consider Geographical differences.

## Findings highlighted above shows an association, not a causality.